## Chassisml Example Notebooks
Welcome to the examples section for [Chassis.ml](https://chassis.ml), which contains notebooks that leverage Chassisml to auto-containerize models built using the most common machine learning frameworks. 

**NOTE:** Chassisml provides two key functionalities: 
1. Create a Docker container from your model code and push that container image to a Docker registry. This is the default behavior.
2. Should you pass valid Modzy credentials as optional parameters, Chassisml will take the container and upload it directly to the Modzy environment you specify. You will notice most of these notebooks deploy the model to one of the Modzy internal development environments.   

Can't find the framework you are looking for or need help? Fork this repository and open a PR, we're always interested in growing this example bank! 

The primary maintainers of Chassis also actively monitor our [Discord Server](https://discord.gg/cHpzY9yCcM), so feel free to join and ask any questions you might have. We'll be there to respond and help out promptly.

In [1]:
import chassisml
import numpy as np
import getpass
import json
from io import StringIO
import pandas as pd
import numpy as np
from sklearn_pmml_model.linear_model import PMMLLinearRegression

## Enter credentials
Dockerhub creds and Modzy API Key

In [2]:
dockerhub_user = getpass.getpass('docker hub username')
dockerhub_pass = getpass.getpass('docker hub password')
modzy_api_key = getpass.getpass('modzy api key')

docker hub username········
docker hub password········
modzy api key········


## Load and Test Model from PMML File

In [3]:
# Prepare data
df = pd.read_csv('data/categorical-test.csv')
Xte = df.iloc[:, 1:]

# Create sample data for testing later
with open("data/sample_regression.csv", "w") as f:
    Xte[:10].to_csv(f, index=False)

In [14]:
# Load model
clf = PMMLLinearRegression(pmml="models/linear-model-lm.pmml")

# Test model
output = clf.predict(Xte)

C:\tools\Anaconda3\envs\chassis-demo\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


## Write process function

* Must take bytes as input
* Preprocess bytes, run inference, postprocess model output, return results

In [23]:
def process(input_bytes):
    # load data
    inputs = pd.read_csv(StringIO(str(input_bytes, "utf-8")))
    
    # predictions
    output = clf.predict(inputs)
    
    # process output
    inference_result = {
        "predictions": [
            {"row": i+1, "score": score.item()} for i, score in enumerate(output)
        ]
    }    
    
    
    # format output
    structured_output = {
        "data": {
            "result": inference_result,
            "explanation": None,
            "drift": None,
        }
    }
    return structured_output

## Initialize Chassis Client
We'll use this to interact with the Chassis service

In [24]:
chassis_client = chassisml.ChassisClient("http://localhost:5000")

## Create and test Chassis model
* Requires `process_fn` defined above

In [25]:
# create Chassis model
chassis_model = chassis_client.create_model(process_fn=process)

# test Chassis model locally (can pass filepath, bufferedreader, bytes, or text here):
sample_filepath = './data/sample_regression.csv'
results = chassis_model.test(sample_filepath)
print(results)

b'{"data":{"result":{"predictions":[{"row":1,"score":0.9590476609154883},{"row":2,"score":0.3066354811397165},{"row":3,"score":0.6510299850602321},{"row":4,"score":0.9548748798188142},{"row":5,"score":0.8892685958181936},{"row":6,"score":0.8744135392564563},{"row":7,"score":0.5529119650921299},{"row":8,"score":0.7939719287317026},{"row":9,"score":0.5676047268515574},{"row":10,"score":0.6944419835236506}]},"explanation":null,"drift":null}}'


C:\tools\Anaconda3\envs\chassis-demo\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


In [26]:
# test environment and model within Chassis service, must pass filepath here:

# dry run before build
test_env_result = chassis_model.test_env(sample_filepath)
print(test_env_result)

Starting test job... Ok!
{'model_output': 'Single input prediction:\n\nb\'{"data":{"result":{"predictions":[{"row":1,"score":0.9590476609154883},{"row":2,"score":0.3066354811397165},{"row":3,"score":0.6510299850602321},{"row":4,"score":0.9548748798188142},{"row":5,"score":0.8892685958181936},{"row":6,"score":0.8744135392564563},{"row":7,"score":0.5529119650921299},{"row":8,"score":0.7939719287317026},{"row":9,"score":0.5676047268515574},{"row":10,"score":0.6944419835236506}]},"explanation":null,"drift":null}}\'\n'}


## Publish model to Modzy
Need to provide model name, model version, Dockerhub credentials, and required Modzy info

In [27]:
MODZY_URL = "https://integration.modzy.engineering/api"

response = chassis_model.publish(
    model_name="PMML Sample Linear Regression",
    model_version="0.0.1",
    registry_user=dockerhub_user,
    registry_pass=dockerhub_pass,
    modzy_sample_input_path=sample_filepath,
    modzy_api_key=modzy_api_key,
    modzy_url=MODZY_URL
)

job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)

Starting build job... Ok!


In [28]:
if chassis_client.get_job_status(job_id)["result"] is not None:
    print("New model URL: {}".format(chassis_client.get_job_status(job_id)["result"]["container_url"]))
else:
    print("Chassis job failed \n\n {}".format(chassis_client.get_job_status(job_id)))

New model URL: https://integration.modzy.engineering/models/tikd0jtz0u/0.0.1


## Run sample job using Modzy SDK
Submit inference job to our newly-deploy model running on Modzy

In [ ]:
from modzy import ApiClient

client = ApiClient(base_url='https://integration.modzy.engineering/api', api_key=modzy_api_key)

input_name = final_status['result']['inputs'][0]['name']
model_id = final_status['result'].get("model").get("modelId")
model_version = final_status['result'].get("version")

inference_job = client.jobs.submit_file(model_id, model_version, {input_name: sample_filepath})
inference_job_result = client.results.block_until_complete(inference_job, timeout=None)
inference_job_results_json = inference_job_result.get_first_outputs()['results.json']
print(inference_job_results_json)